# U.S. Medical Insurance Costs

## Introduction

This project explores issues within a provided dataset on US insurance costs. The data is supplied is a CSV format, and to give an indicative view of what this data looks like, the first 5 entries in the CSV file are shown below:

In [49]:
# Set up libraries, import csv, and print first 5 rows
import numpy as np
import pandas as pd
import os

with open("insurance.csv") as insurance_csv_file:
  pd_insurance_csv = pd.read_csv(insurance_csv_file)
pd_insurance_csv.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


There are 1338 subjects in the dataset.

## Scope

The aims of the project are to explore:

 - Frequency of region of people 
 - Two-way table between male and female, and smoking and non-smoking, with a Chi squared analysis to test whether there is statistical significance
 - Is there a difference in ages between areas?
 - Is there a relationship between cost of insurance, and area lived in?


In [30]:
# Manual calculation of regions frequency table

region_list = []
region_count = {}

with open("insurance.csv") as insurance_csv_file:
    insurance_csv = list(csv.reader(insurance_csv_file))

  

for person in insurance_csv:
    region_list.append(person[5])
      
for item in region_list:
    if item not in region_count:
        region_count[item] = region_list.count(item)

del region_count["region"]

print(region_count)
total = len(region_list)

for value in region_count.values():
    print(str(round((value/total*100), 1)) + "%")
    

{'southwest': 325, 'southeast': 364, 'northwest': 325, 'northeast': 324}
24.3%
27.2%
24.3%
24.2%


In [47]:
# Create Pandas csv reader, and create region frequency tab;e from it

region_summary = pd.crosstab(index=pd_insurance_csv["region"],  # Make a crosstab
                              columns="count")      # Name the count column

region_summary

col_0,count
region,
northeast,324
northwest,325
southeast,364
southwest,325


In [37]:
# Male/female breakdown
sex_summary = pd.crosstab(index=pd_insurance_csv["sex"],  # Make a crosstab
                              columns="count")      # Name the count column

sex_summary

col_0,count
sex,
female,662
male,676


In [38]:
sex_summary = pd.crosstab(index=pd_insurance_csv["children"],  # Make a crosstab
                              columns="count")      # Name the count column

sex_summary

col_0,count
children,
0,574
1,324
2,240
3,157
4,25
5,18


In [39]:
sex_summary = pd.crosstab(index=pd_insurance_csv["smoker"],  # Make a crosstab
                              columns="count")      # Name the count column

sex_summary

col_0,count
smoker,
no,1064
yes,274


Two-way tables

In [42]:
smoking_sex = pd.crosstab(index=pd_insurance_csv["smoker"], 
                           columns=pd_insurance_csv["sex"])

smoking_sex.index= ["no","yes"]

smoking_sex

sex,female,male
no,547,517
yes,115,159


In [65]:
pd_insurance_csv.groupby(["region", "sex"])["charges"].mean()

region     sex   
northeast  female    12953.203151
           male      13854.005374
northwest  female    12479.870397
           male      12354.119575
southeast  female    13499.669243
           male      15879.617173
southwest  female    11274.411264
           male      13412.883576
Name: charges, dtype: float64

In [70]:
bins = [0,10,20,30,40,50,60,70,80,90]
labels = ["0-9", "10-19", "20-29", "30-29", "40-49", "50-59", "60-69", "70-79", "80-89", "90-99"]

In [71]:
pd_insurance_csv["Age Table"] = pd.cut(pd_insurance_csv["age"], bins, labels=labels)
pd_insurance_csv.head(25)

ValueError: Bin labels must be one fewer than the number of bin edges